Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
IMAGE_SIZE = 28
NUM_LABELS = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, IMAGE_SIZE * IMAGE_SIZE)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(NUM_LABELS) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# L2 regularization on logistic model
# no real purpose.  it was already done.  maybe i should do it in tensorflow

from sklearn.linear_model import LogisticRegression

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_labels_for_logistic = save['train_labels']
  valid_labels_for_logistic = save['valid_labels']
  test_labels_for_logistic = save['test_labels']

SAMPLE_SIZE = 500
mini_train_dataset = train_dataset[100:SAMPLE_SIZE+100]
mini_train_labels = train_labels_for_logistic[100:SAMPLE_SIZE+100]

print("train_dataset", train_dataset.shape)
print("mini_train_dataset", mini_train_dataset.shape)
print("train_labels", train_labels_for_logistic.shape)
print("mini_train_labels", mini_train_labels.shape)

model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
                           class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='ovr', 
                           verbose=0, warm_start=False, n_jobs=-1)
model.fit(mini_train_dataset, mini_train_labels)

test_dataset_size = test_dataset.shape[0]
prediction = model.predict(test_dataset.reshape(test_dataset_size,IMAGE_SIZE*IMAGE_SIZE))
correct = np.sum(prediction == test_labels_for_logistic)
print(correct, 'correct predictions of', test_dataset_size, 'for', 100.0*correct/test_dataset_size, '%')


train_dataset (200000, 784)
mini_train_dataset (500, 784)
train_labels (200000,)
mini_train_labels (500,)
8381 correct predictions of 10000 for 83.81 %


In [7]:
# L2 regularization on neural network model

BATCH_SIZE = 128

def forward_prop(dataset, weights1, biases1, weights2, biases2):
  calc1_output = tf.matmul(dataset, weights1) + biases1
  relu_output = tf.nn.relu(calc1_output)
  return tf.matmul(relu_output, weights2) + biases2

graph5 = tf.Graph()
with graph5.as_default():

  # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.as_dtype(train_dataset.dtype), shape = (BATCH_SIZE, train_dataset.shape[1]))
  tf_train_labels = tf.placeholder(tf.as_dtype(train_labels.dtype), shape = (BATCH_SIZE, train_labels.shape[1]))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  input_size = train_dataset.shape[1]
  hidden_size = 1028
  output_size = train_labels.shape[1]
  weights1 = tf.Variable(tf.truncated_normal([input_size, hidden_size]))
  biases1 = tf.Variable(tf.truncated_normal([hidden_size], stddev=1.0))
  weights2 = tf.Variable(tf.truncated_normal([hidden_size, output_size]))
  biases2 = tf.Variable(tf.truncated_normal([output_size], stddev=1.0))

  # Training computation.
  logits = forward_prop(tf_train_dataset, weights1, biases1, weights2, biases2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
  loss += 5e-4 * regularizers
  loss_summary = tf.scalar_summary("loss", loss)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2))

print('network defined')

num_training_examples = train_dataset.shape[0]

with tf.Session(graph=graph5) as session:
  tf.initialize_all_variables().run()
  
  for step in range(3001):
    offset = (step * BATCH_SIZE) % (num_training_examples - BATCH_SIZE)

    batch_dataset = train_dataset[offset:(offset+BATCH_SIZE), :]
    batch_labels = train_labels[offset:(offset+BATCH_SIZE), :]

    feed_dict = {tf_train_dataset: batch_dataset, tf_train_labels: batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if step % 500 == 0:
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels), "\n")

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

network defined
Minibatch loss at step 0: 543.508667
Minibatch accuracy: 12.5%
Validation accuracy: 29.1% 

Minibatch loss at step 500: 155.223480
Minibatch accuracy: 87.5%
Validation accuracy: 81.0% 

Minibatch loss at step 1000: 138.165207
Minibatch accuracy: 81.2%
Validation accuracy: 82.1% 

Minibatch loss at step 1500: 120.085419
Minibatch accuracy: 89.1%
Validation accuracy: 83.5% 

Minibatch loss at step 2000: 108.651634
Minibatch accuracy: 84.4%
Validation accuracy: 82.8% 

Minibatch loss at step 2500: 96.588432
Minibatch accuracy: 85.2%
Validation accuracy: 84.0% 

Minibatch loss at step 3000: 86.730339
Minibatch accuracy: 82.8%
Validation accuracy: 83.6% 

Test accuracy: 90.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [11]:
# Restrict to 10 batches for extreme overfitting

BATCH_SIZE = 128
SAMPLE_SIZE = 10*BATCH_SIZE
mini_train_dataset = train_dataset[:SAMPLE_SIZE]
mini_train_labels = train_labels[:SAMPLE_SIZE]

def forward_prop(dataset, weights1, biases1, weights2, biases2):
  calc1_output = tf.matmul(dataset, weights1) + biases1
  relu_output = tf.nn.relu(calc1_output)
  return tf.matmul(relu_output, weights2) + biases2

graph5 = tf.Graph()
with graph5.as_default():

  # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.as_dtype(mini_train_dataset.dtype), shape = (BATCH_SIZE, mini_train_dataset.shape[1]))
  tf_train_labels = tf.placeholder(tf.as_dtype(mini_train_labels.dtype), shape = (BATCH_SIZE, mini_train_labels.shape[1]))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  input_size = mini_train_dataset.shape[1]
  hidden_size = 1028
  output_size = mini_train_labels.shape[1]
  weights1 = tf.Variable(tf.truncated_normal([input_size, hidden_size]))
  biases1 = tf.Variable(tf.truncated_normal([hidden_size], stddev=1.0))
  weights2 = tf.Variable(tf.truncated_normal([hidden_size, output_size]))
  biases2 = tf.Variable(tf.truncated_normal([output_size], stddev=1.0))

  # Training computation.
  logits = forward_prop(tf_train_dataset, weights1, biases1, weights2, biases2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
  loss += 5e-4 * regularizers
  loss_summary = tf.scalar_summary("loss", loss)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2))

print('network defined')

num_training_examples = mini_train_dataset.shape[0]

with tf.Session(graph=graph5) as session:
  tf.initialize_all_variables().run()
  
  for step in range(3001):
    offset = (step * BATCH_SIZE) % (num_training_examples - BATCH_SIZE)

    batch_dataset = mini_train_dataset[offset:(offset+BATCH_SIZE), :]
    batch_labels = mini_train_labels[offset:(offset+BATCH_SIZE), :]

    feed_dict = {tf_train_dataset: batch_dataset, tf_train_labels: batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if step % 500 == 0:
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels), "\n")

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

network defined
Minibatch loss at step 0: 611.137451
Minibatch accuracy: 9.4%
Validation accuracy: 23.0% 

Minibatch loss at step 500: 142.993271
Minibatch accuracy: 100.0%
Validation accuracy: 76.4% 

Minibatch loss at step 1000: 129.384903
Minibatch accuracy: 100.0%
Validation accuracy: 76.4% 

Minibatch loss at step 1500: 117.071701
Minibatch accuracy: 100.0%
Validation accuracy: 76.4% 

Minibatch loss at step 2000: 105.930176
Minibatch accuracy: 100.0%
Validation accuracy: 76.4% 

Minibatch loss at step 2500: 95.849335
Minibatch accuracy: 100.0%
Validation accuracy: 76.4% 

Minibatch loss at step 3000: 86.727486
Minibatch accuracy: 100.0%
Validation accuracy: 76.4% 

Test accuracy: 84.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [38]:
# Introduce dropout

import random

BATCH_SIZE = 128
SAMPLE_SIZE = 10*BATCH_SIZE
# mini_train_dataset = train_dataset[:SAMPLE_SIZE]
# mini_train_labels = train_labels[:SAMPLE_SIZE]

def forward_prop(dataset, train=False):
  layer1 = tf.matmul(dataset, weights1) + biases1
  keep_prob = 0.5 if train else 1.0
  relu = tf.nn.dropout(tf.nn.relu(layer1), keep_prob)
  return tf.matmul(relu, weights2) + biases2


def loss():
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
#   loss += 5e-4 * regularizers
  return loss

graph5 = tf.Graph()
with graph5.as_default():

  # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.as_dtype(train_dataset.dtype), shape = (BATCH_SIZE, train_dataset.shape[1]))
  tf_train_labels = tf.placeholder(tf.as_dtype(train_labels.dtype), shape = (BATCH_SIZE, train_labels.shape[1]))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Weights & Biases
  input_size = train_dataset.shape[1]
  hidden_size = 1028
  output_size = train_labels.shape[1]
  weights1 = tf.Variable(tf.truncated_normal([input_size, hidden_size]))
  biases1 = tf.Variable(tf.truncated_normal([hidden_size], stddev=1.0))
  weights2 = tf.Variable(tf.truncated_normal([hidden_size, output_size]))
  biases2 = tf.Variable(tf.truncated_normal([output_size], stddev=1.0))

  # Training computation.
  logits = forward_prop(tf_train_dataset, train=True)
  loss = loss()

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

print('network defined')

num_training_examples = mini_train_dataset.shape[0]

with tf.Session(graph=graph5) as session:
  tf.initialize_all_variables().run()
  
  for step in range(3001):
    offset = (step % 10 * BATCH_SIZE) % (num_training_examples - BATCH_SIZE)

    batch_dataset = train_dataset[offset:(offset+BATCH_SIZE), :]
    batch_labels = train_labels[offset:(offset+BATCH_SIZE), :]

    feed_dict = {tf_train_dataset: batch_dataset, tf_train_labels: batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if step % 500 == 0:
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels), "\n")

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

network defined
Minibatch loss at step 0: 516.165771
Minibatch accuracy: 13.3%
Validation accuracy: 31.0% 

Minibatch loss at step 500: 0.161151
Minibatch accuracy: 98.4%
Validation accuracy: 79.8% 

Minibatch loss at step 1000: 0.299517
Minibatch accuracy: 99.2%
Validation accuracy: 79.9% 

Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.0% 

Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.5% 

Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.2% 

Minibatch loss at step 3000: 0.312540
Minibatch accuracy: 99.2%
Validation accuracy: 80.4% 

Test accuracy: 87.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
